Alex Thornton (apt2141)  
Columbia University Fall 2021  
ELEN E6893 - Big Data Analytics  
Research Project: SpotifyClassifier

1. Connect to GCP

In [1]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project spotifyclassifier
!pip install spotipy

Updated property [core/project].


In [2]:
import os
os.environ['SPOTIPY_CLIENT_ID'] = 'SECRET'
os.environ['SPOTIPY_CLIENT_SECRET'] = 'SECRET'

In [3]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')
from apiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload

3. Import Packages

In [13]:
import requests
import json
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials

3. Setup Spotify API

In [5]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

4. Do Stuff

In [6]:
genres = ["alternative","indie","alt-rock","punk","grunge","punk-rock","indie-pop","emo","goth",\
    "hardcore","death-metal","grindcore","black-metal","heavy-metal","metalcore","metal","metal-misc",\
    "dance","electronic","electro","edm","garage","house","chicago-house","breakbeat","dubstep","techno","j-dance","deep-house",\
    "ambient","chill","sleep","new-age","study","piano",\
    "rock","rock-n-roll","rockabilly","j-rock","hard-rock","psych-rock","power-pop","guitar","drum-and-bass",\
    "country","bluegrass","acoustic","folk","honky-tonk",\
    "pop","k-pop","j-pop","cantopop","party","pop-film","r-n-b","club","hip-hop",\
    "jazz","soul","funk","blues","afrobeat",\
    "latin","latino","reggaeton","samba","salsa","bossanova",\
    "classical","opera",\
    "show-tunes","disney"]

!mkdir /track-files
for genre in genres:
    with open('/track-files/'+genre+'.json', 'wb') as f:
        request = gcs_service.objects().get_media(bucket='track-lists-full',object=genre+'.json')
        media = MediaIoBaseDownload(f, request)
        done = False
        while not done:
            _, done = media.next_chunk()

    with open('/track-files/'+genre+'.json', "r") as tracklist:
        tracklist = json.load(tracklist)
        num_songs = len(tracklist.keys())
    print("Genre: "+genre+", # Tracks: "+str(num_songs))



Genre: alternative, # Tracks: 500
Genre: indie, # Tracks: 500
Genre: alt-rock, # Tracks: 497
Genre: punk, # Tracks: 500
Genre: grunge, # Tracks: 500
Genre: punk-rock, # Tracks: 500
Genre: indie-pop, # Tracks: 100
Genre: emo, # Tracks: 500
Genre: goth, # Tracks: 500
Genre: hardcore, # Tracks: 500
Genre: death-metal, # Tracks: 500
Genre: grindcore, # Tracks: 500
Genre: black-metal, # Tracks: 500
Genre: heavy-metal, # Tracks: 500
Genre: metalcore, # Tracks: 500
Genre: metal, # Tracks: 100
Genre: metal-misc, # Tracks: 100
Genre: dance, # Tracks: 494
Genre: electronic, # Tracks: 500
Genre: electro, # Tracks: 500
Genre: edm, # Tracks: 105
Genre: garage, # Tracks: 500
Genre: house, # Tracks: 500
Genre: chicago-house, # Tracks: 500
Genre: breakbeat, # Tracks: 500
Genre: dubstep, # Tracks: 500
Genre: techno, # Tracks: 500
Genre: j-dance, # Tracks: 369
Genre: deep-house, # Tracks: 500
Genre: ambient, # Tracks: 500
Genre: chill, # Tracks: 500
Genre: sleep, # Tracks: 500
Genre: new-age, # Tracks: 

In [7]:
!ls /track-files

acoustic.json	    disney.json		house.json	 power-pop.json
afrobeat.json	    drum-and-bass.json	indie.json	 psych-rock.json
alternative.json    dubstep.json	indie-pop.json	 punk.json
alt-rock.json	    edm.json		jazz.json	 punk-rock.json
ambient.json	    electro.json	j-dance.json	 reggaeton.json
black-metal.json    electronic.json	j-pop.json	 r-n-b.json
bluegrass.json	    emo.json		j-rock.json	 rockabilly.json
blues.json	    folk.json		k-pop.json	 rock.json
bossanova.json	    funk.json		latin.json	 rock-n-roll.json
breakbeat.json	    garage.json		latino.json	 salsa.json
cantopop.json	    goth.json		metalcore.json	 samba.json
chicago-house.json  grindcore.json	metal.json	 show-tunes.json
chill.json	    grunge.json		metal-misc.json  sleep.json
classical.json	    guitar.json		new-age.json	 soul.json
club.json	    hardcore.json	opera.json	 study.json
country.json	    hard-rock.json	party.json	 techno.json
dance.json	    heavy-metal.json	piano.json
death-metal.json    hip-hop.json	pop-film.js

In [8]:
!mkdir /audio-features

In [9]:
for genre in genres:
    !mkdir /audio-features/$genre

In [10]:
#Map subgenres to their supergenre parent node
supergenre = {
    'alternative':'alternative',
    'indie':'alternative',
    'alt-rock':'alternative',
    'punk':'alternative',
    'grunge':'alternative',
    'punk-rock':'alternative',
    'indie-pop':'alternative',
    'emo':'alternative',
    'goth':'alternative',
    'hardcore':'hardcore',
    'death-metal':'hardcore',
    'grindcore':'hardcore',
    'black-metal':'hardcore',
    'heavy-metal':'hardcore',
    'metalcore':'hardcore',
    'metal':'hardcore',
    'metal-misc':'hardcore',
    'dance':'dance',
    'electronic':'dance',
    'electro':'dance',
    'edm':'dance',
    'garage':'dance',
    'house':'dance',
    'chicago-house':'dance',
    'breakbeat':'dance',
    'dubstep':'dance',
    'techno':'dance',
    'j-dance':'dance',
    'deep-house':'dance',
    'ambient':'easy-listening',
    'chill':'easy-listening',
    'sleep':'easy-listening',
    'new-age':'easy-listening',
    'study':'easy-listening',
    'piano':'easy-listening',
    'rock':'rock',
    'rock-n-roll':'rock',
    'rockabilly':'rock',
    'j-rock':'rock',
    'hard-rock':'rock',
    'psych-rock':'rock',
    'power-pop':'rock',
    'guitar':'rock',
    'drum-and-bass':'rock',
    'country':'country',
    'bluegrass':'country',
    'acoustic':'country',
    'folk':'country',
    'honky-tonk':'country',
    'jazz':'jazz',
    'soul':'jazz',
    'funk':'jazz',
    'blues':'jazz',
    'afrobeat':'jazz',
    'latin':'latin',
    'latino':'latin',
    'reggaeton':'latin',
    'samba':'latin',
    'salsa':'latin',
    'bossanova':'latin',
    'classical':'classical',
    'opera':'classical',
    'disney':'musical',
    'show-tunes':'musical'
}

In [19]:
for genre in genres:
    print("Starting " +genre)
    with open('/track-files/'+genre+'.json', "r") as tracklist:
        tracklist = json.load(tracklist)
        for track in tracklist:
            track_id = tracklist[track]
            track_data = sp.audio_features(track_id)
            track_data = track_data[0]

            # Save audio features locally
            with open("/audio-features/"+track_id+".json", 'w') as f:
                #Add some of our own labels
                track_data['subgenre-label'] = genre 
                track_data['super-genre-label'] = supergenre[genre]
                track_data['song-title'] = track

                #Fetch some additional info for us
                basic_info = sp.track(track_id)
                track_data['popularity'] = basic_info['popularity']
                track_data['explicit'] = basic_info['explicit']
                album = basic_info['album']
                track_data['release_date'] = album['release_date']
                track_data['release_date_precision'] = album['release_date_precision']

                #Add in some Spotify Audio Analysis Vectors
                analysis = sp.audio_analysis(track_id)

                num_seg = 0
                pitches = np.zeros(12)
                timbre = np.zeros(12)

                analysis_data = analysis['track']
                time_signature = analysis_data['time_signature']
                key = analysis_data['key']
                mode = analysis_data['mode']

                if "segments" in analysis:
                    for _, j in enumerate(analysis['segments']):
                        pitches += np.array(j['pitches'])
                        timbre += np.array(j['timbre'])
                        num_seg+=1

                track_data['avg_pitches'] = list(pitches/num_seg)
                track_data['avg_timbre'] = list(timbre/num_seg)
                #print(track_data)

                json.dump(track_data, f, indent=4)

            # Upload audio features to GCP bucket
            media = MediaFileUpload("/audio-features/"+track_id+".json", 
                              mimetype='application/json',
                              resumable=True)

            request = gcs_service.objects().insert(bucket='audio-features-fullset', 
                                                  name=genre+"/"+track_id+".json",
                                                  media_body=media)

            response = None
            while response is None:
                _, response = request.next_chunk()

{'danceability': 0.795, 'energy': 0.706, 'key': 7, 'loudness': -7.494, 'mode': 1, 'speechiness': 0.101, 'acousticness': 0.0172, 'instrumentalness': 0.000384, 'liveness': 0.326, 'valence': 0.623, 'tempo': 95.593, 'type': 'audio_features', 'id': '3oVSsJXNADnYAnZWiEWFJH', 'uri': 'spotify:track:3oVSsJXNADnYAnZWiEWFJH', 'track_href': 'https://api.spotify.com/v1/tracks/3oVSsJXNADnYAnZWiEWFJH', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3oVSsJXNADnYAnZWiEWFJH', 'duration_ms': 171067, 'time_signature': 4, 'subgenre-label': 'alternative', 'super-genre-label': 'alternative', 'song-title': 'What I Got', 'popularity': 0, 'explicit': True, 'release_date': '1996-07-30', 'release_date_precision': 'day', 'avg_pitches': [0.5294890885750954, 0.6355430038510912, 0.519951219512195, 0.2480038510911425, 0.26212965340179695, 0.2443106546854943, 0.3504159178433889, 0.38361617458279845, 0.3120962772785621, 0.3722143774069321, 0.2616854942233632, 0.3376469833119385], 'avg_timbre': [46.5841720154

KeyboardInterrupt: ignored